In [ ]:
##############################################################################
# FULL SCRIPT WITH REGRESSION, ANOVA, TUKEY’S HSD, AND SAVING SEPARATE CSVs
##############################################################################

import os
import pandas as pd
import numpy as np

# Statsmodels-related imports
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.multicomp import pairwise_tukeyhsd

##############################################################################
# Step 0: File paths (adjust as needed)
##############################################################################
file_path = r"your DB path"
output_folder = r"your output folder path"

# Ensure output folder exists
os.makedirs(output_folder, exist_ok=True)

##############################################################################
# Step 1: Read the dataset
##############################################################################
merged_data = pd.read_csv(file_path)

# Make sure columns exist:
#   'age_at_mri', 'ca_delta', 'sex_x',
#   'Visceral_adipose_tissue_volume', 'Subcutaneous_fat_body',
#   'Proton_density_fat_fraction_liver', 'Muscle_fat_infiltration',
#   'Total_trunk_fat_volume', 'Summed_MET_minutes_p_w',
#   'Weight_Group', 'BMI'

##############################################################################
# Step 2: Create Fitness / Fatness categories
##############################################################################
# 2a) Basic 'Unfit' vs. 'Fit' using MET threshold
merged_data['Fitness'] = np.where(
    merged_data['Summed_MET_minutes_p_w'] >= 1000, 
    'Fit', 
    'Unfit'
)

# 2b) Fatness category (Normal Weight, Overweight, Obese)
merged_data['Fatness'] = pd.cut(
    merged_data['BMI'],
    bins=[0, 25, 30, np.inf],
    labels=['Normal Weight', 'Overweight', 'Obese']
)

# -------------------------------------------------------------------------
# Rename them so "Unfit" sorts before "Fit":
#   'Unfit' → 'AUnfit'
#   'Fit'   → 'BFit'
# -------------------------------------------------------------------------
merged_data['Fitness'] = merged_data['Fitness'].replace({
    'Unfit': 'AUnfit',
    'Fit': 'BFit'
})

# 2c) Combined category for ANOVA/Tukey: "AUnfit-Obese", "BFit-Overweight", etc.
merged_data['Fitness_Fatness'] = (
    merged_data['Fitness'] + '-' + merged_data['Fatness'].astype(str)
)

##############################################################################
# Step 3: Multiple Linear Regression (Full Data)
#    ca_delta ~ age + sex + BMI + MET + Visceral_adipose_tissue_volume
##############################################################################
model_formula = """
    ca_delta ~ age_at_mri + C(sex_x) + BMI + Summed_MET_minutes_p_w 
               + Visceral_adipose_tissue_volume
"""
model = smf.ols(model_formula, data=merged_data).fit()
print("=== Multiple Linear Regression Results ===")
print(model.summary())

##############################################################################
# Step 4: Subgroup Analysis for Fit–Obese (now "BFit-Obese")
##############################################################################
fit_obese_data = merged_data[
    (merged_data['Fitness'] == 'BFit') & 
    (merged_data['Fatness'] == 'Obese')
]

subgroup_formula = "ca_delta ~ age_at_mri + C(sex_x) + Visceral_adipose_tissue_volume"
subgroup_model = smf.ols(subgroup_formula, data=fit_obese_data).fit()
print("\n=== Subgroup Analysis (Fit–Obese) ===")
print(subgroup_model.summary())

##############################################################################
# Step 5: One-Way ANOVA of ca_delta across the combined categories
##############################################################################
# Fit a one-way ANOVA model using categories in "Fitness_Fatness"
model_anova = smf.ols("ca_delta ~ C(Fitness_Fatness)", data=merged_data).fit()
anova_result = sm.stats.anova_lm(model_anova, typ=2)

print("\n=== One-Way ANOVA Results (Fitness_Fatness) ===")
print(anova_result)

##############################################################################
# Step 6: Tukey’s HSD for Pairwise Comparisons Among All Groups
##############################################################################
tukey_result = pairwise_tukeyhsd(
    endog=merged_data['ca_delta'],
    groups=merged_data['Fitness_Fatness'],
    alpha=0.05
)

print("\n=== Full Tukey’s HSD Pairwise Comparisons (All Groups) ===")
print(tukey_result)

##############################################################################
# Step 7: Convert the Tukey’s HSD results to a DataFrame
#         Then keep only rows where group1 is "AUnfit-..." and group2 is "BFit-..."
##############################################################################
tukey_summary = tukey_result.summary()
tukey_data = tukey_summary.data[1:]  # skip header
tukey_cols = tukey_summary.data[0]   # header row

df_tukey = pd.DataFrame(tukey_data, columns=tukey_cols)

# B) Build a dictionary of mean ca_delta for each category
group_means = (
    merged_data.groupby('Fitness_Fatness')['ca_delta']
    .mean()
    .to_dict()
)

# C) Add columns for each group’s mean ca_delta
df_tukey['group1_mean_ca_delta'] = df_tukey['group1'].map(group_means)
df_tukey['group2_mean_ca_delta'] = df_tukey['group2'].map(group_means)

# D) Filter: keep only group1 = AUnfit-..., group2 = BFit-...
mask = (
    df_tukey['group1'].str.startswith('AUnfit') & 
    df_tukey['group2'].str.startswith('BFit')
)
df_filtered = df_tukey[mask].copy()

print("\n=== Filtered Tukey Comparisons (group1=Unfit, group2=Fit) ===")
print(df_filtered)

##############################################################################
# Step 8: (Optional) Remove the leading “A” or “B” from final display
##############################################################################
def remove_leading_letter(x):
    if x.startswith('A') or x.startswith('B'):
        return x[1:]
    return x

df_filtered['group1'] = df_filtered['group1'].apply(remove_leading_letter)
df_filtered['group2'] = df_filtered['group2'].apply(remove_leading_letter)

print("\n=== Final Table (group1=Unfit..., group2=Fit...) with mean ca_delta ===")
print(df_filtered)

##############################################################################
# Step 9: SAVE Outputs
##############################################################################

# 9a) Save regression summaries
with open(os.path.join(output_folder, 'model_summary.txt'), 'w') as f:
    f.write(model.summary().as_text())

with open(os.path.join(output_folder, 'model_summary_fit_obese.txt'), 'w') as f:
    f.write(subgroup_model.summary().as_text())

# 9b) Save ANOVA & Tukey (full) results (as text)
with open(os.path.join(output_folder, 'anova_results.txt'), 'w') as f:
    f.write(str(anova_result))

with open(os.path.join(output_folder, 'tukey_full.txt'), 'w') as f:
    f.write(str(tukey_result))

# 9c) Save the full Tukey table to CSV (# NEW)
df_tukey.to_csv(os.path.join(output_folder, 'tukey_full_table.csv'), index=False)

# 9d) Save the final filtered table to CSV (# NEW)
df_filtered.to_csv(os.path.join(output_folder, 'tukey_filtered_unfit_vs_fit.csv'), index=False)
